# Feature engineering : Create Technical indicator features

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import io , os , time ,requests,json
from time import sleep
import bs4
# import lxml.etree as xml
# import starfishX as sx
import pandas_datareader as pdr
from yahoo_finance import Share
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# init 
datapath = '/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice'

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# create list of S&P500 symbols
symbol = pd.read_csv(os.path.join(datapath,"sp500symbol.csv"))
symbol_list = symbol['Symbol'].tolist()

for sym in enumerate(symbol_list):
    if '.' in sym[1]:
        del symbol_list[sym[0]]

In [4]:
df = pd.read_csv(os.path.join(datapath,"histprice2.csv"))
df.head()

,Unnamed: 0,Date,Open,High,Low,Close,Adj Close,Volume,Name,industry
0,0,2014-12-31,28.205000,28.282499,27.552500,27.594999,25.137676,165613600.0,AAPL,Consumer Electronics
1,1,2015-01-02,27.847500,27.860001,26.837500,27.332500,24.898552,212818400.0,AAPL,Consumer Electronics
2,2,2015-01-05,27.072500,27.162500,26.352501,26.562500,24.197117,257142000.0,AAPL,Consumer Electronics
3,3,2015-01-06,26.635000,26.857500,26.157499,26.565001,24.199400,263188400.0,AAPL,Consumer Electronics
4,4,2015-01-07,26.799999,27.049999,26.674999,26.937500,24.538729,160423600.0,AAPL,Consumer Electronics


RSI Indicator

In [19]:
new_df.head()

,High,Low,Open,Close,Volume,Adj Close,ema10,ema20,ema75,macd,macd_signal_line,rsi,L14,H14,%K,%D,name
Date,,,,,,,,,,,,,,,,,
2014-12-30,15.24,14.95,15.12,15.18,695000.0,13.972668,13.972668,13.972668,13.972668,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NWS
2014-12-31,15.32,15.07,15.20,15.08,644100.0,13.880621,13.955932,13.963901,13.970245,-0.007343,-0.001469,NaN,NaN,NaN,NaN,NaN,NWS
2015-01-02,15.28,14.80,15.13,14.98,905000.0,13.788574,13.925503,13.947204,13.965465,-0.020355,-0.005246,NaN,NaN,NaN,NaN,NaN,NWS
2015-01-05,14.91,14.53,14.91,14.64,1018500.0,13.475616,13.843705,13.902290,13.952574,-0.055283,-0.015253,NaN,NaN,NaN,NaN,NaN,NWS
2015-01-06,14.67,14.32,14.65,14.47,1197100.0,13.319136,13.748329,13.846752,13.935904,-0.094501,-0.031103,NaN,NaN,NaN,NaN,NaN,NWS


In [9]:
# Create Function RSI Statistic  (Simple moving average)
def RSIfun(df, n=14):
    delta = df['Adj Close'].diff()
    #-----------
    # Make the positive gains (up) and negative gains (down) Series
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    dUp= delta[delta > 0]
    dDown= delta[delta < 0]

    # Calculate the SMA
    roll_up = up.rolling(n).mean()
    roll_down = down.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))

    return rsi

In [10]:
# Create Function RSI Statistic  (Exponential Moving Average)
def RSI_ewma(df, n=14):
    delta = df['Adj Close'].diff()
    #-----------
    # Make the positive gains (up) and negative gains (down) Series
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    dUp= delta[delta > 0]
    dDown= delta[delta < 0]

    # Calculate the EWMA
    roll_up_ewma = up.ewm(span=n,adjust=False).mean()
    roll_ewma = down.abs().ewm(span=n,adjust=False).mean()


    # Calculate the RSI based on EWMA
    rs_ewma = roll_up_ewma / roll_down_ewma
    rsi_ewma = 100.0 - (100.0 / (1.0 + rs_ewma))

    return rsi_ewma

In [11]:
startdate='2014-12-31'
#enddate='2020-09-15'

In [34]:
new_df=pd.DataFrame()
count = 0
for i,val in enumerate(symbol_list):
    try:
        val=str(val)
        print(i,val)
        new_df=pdr.DataReader(val,start=startdate,data_source='yahoo')

        #Create New Feature Day Month Year DayEnglish
        #new_df= get_X_y(new_df)
        
        #new_df= one_hot_encoder(new_df)

        #Create New Column EMA10,20,75
        new_df['ema10']= new_df['Adj Close'].ewm(span=10, adjust=False).mean()
        new_df['ema20']= new_df['Adj Close'].ewm(span=20, adjust=False).mean()
        new_df['ema75']= new_df['Adj Close'].ewm(span=75, adjust=False).mean()
        
        #Create New Column MACD
        exp1 = new_df['Adj Close'].ewm(span=12, adjust=False).mean()
        exp2 = new_df['Adj Close'].ewm(span=26, adjust=False).mean()
        macd = exp1-exp2
        new_df['macd'] = macd
        new_df['macd_signal_line']= macd.ewm(span=9, adjust=False).mean()
        
        # Create New Columns RSI 
        new_df['rsi'] =RSIfun(new_df,n=14)
        #new_df['rsi_ewma'] =RSI_ewma(new_df,n=14)

        #Create Stocastice Ossilator
        new_df['L14'] = new_df['Low'].rolling(window=14).min()
        #Create the "H14" column in the DataFrame
        new_df['H14'] = new_df['High'].rolling(window=14).max()
        #Create the "%K" column in the DataFrame
        new_df['%K'] = 100*((new_df['Close'] - new_df['L14']) / (new_df['H14'] - new_df['L14']) )
        #Create the "%D" column in the DataFrame
        new_df['%D'] = new_df['%K'].rolling(window=3).mean()

        #Shift Price 1day before
        #new_df['Adj Close-1day']=new_df['Adj Close'].shift(periods=1,axis=0) 
        #new_df['Adj Close-5day']=new_df['Adj Close'].shift(periods=5,axis=0)

        #Crea New column Stock Name
        new_df['name']=val
        new_df.to_csv(val+".csv")

        count +=1
        if count%40 == 0:
            sleep(5)

    except Exception:
        None
        

0 AAPL
1 MSFT
2 AMZN
3 FB
4 GOOGL
5 GOOG
6 TSLA
7 BRK.B
8 JNJ
9 JPM
10 V
11 PG
12 NVDA
13 UNH
14 DIS
15 HD
16 MA
17 PYPL
18 VZ
19 ADBE
20 BAC
21 CMCSA
22 NFLX
23 PFE
24 CRM
25 KO
26 T
27 MRK
28 PEP
29 WMT
30 INTC
31 ABT
32 CSCO
33 ABBV
34 TMO
35 NKE
36 XOM
37 AVGO
38 QCOM
39 CVX
40 ACN
41 COST
42 MCD
43 MDT
44 TXN
45 HON
46 NEE
47 DHR
48 BMY
49 UNP
50 LIN
51 LLY
52 AMGN
53 PM
54 C
55 WFC
56 ORCL
57 UPS
58 SBUX
59 LOW
60 BA
61 IBM
62 AMD
63 RTX
64 NOW
65 MMM
66 BLK
67 INTU
68 MS
69 CAT
70 AMT
71 ISRG
72 GE
73 CHTR
74 CVS
75 GS
76 TGT
77 LMT
78 BKNG
79 DE
80 FIS
81 MDLZ
82 TJX
83 MO
84 SCHW
85 MU
86 SYK
87 TMUS
88 AMAT
89 ANTM
90 AXP
91 ZTS
92 SPGI
93 ADP
94 CL
95 GILD
96 CI
97 PLD
98 BDX
99 ATVI
100 CSX
101 CB
102 LRCX
103 ADSK
104 CCI
105 DUK
106 FDX
107 CME
108 USB
109 TFC
110 ICE
111 SO
112 EQIX
113 PNC
114 FISV
115 VRTX
116 D
117 GPN
118 SHW
119 NSC
120 APD
121 EL
122 ITW
123 MMC
124 PGR
125 EW
126 HUM
127 ILMN
128 GM
129 ECL
130 ADI
131 DG
132 REGN
133 DD
134 BSX
135 NEM
136 EMR
13

In [33]:
new_df_set100=pd.DataFrame()

for i in symbol_list:
    df=pd.read_csv("/mnt/c/Users/poom/drive/01project/01cap_stone/04data/01stockprice/02sp500/"+i+".csv")
    new_df_set100=new_df_set100.append(df)

new_df_set100.dropna(how= 'any').to_csv(os.path.join(datapath,'histprice3.csv'),index_label = False)
new_df_set100.shape

(747374, 18)

In [1]:
len(new_df_set100['name'].unique())

NameError: name 'new_df_set100' is not defined